<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import plotly.express as px

In [3]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select count(*) from vacancies
'''

In [5]:
pd.read_sql_query(query_3_1, connection)

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query = f'''select count(*) from employers
'''

In [7]:
# результат запроса
pd.read_sql_query(query, connection)

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3 = f'''select count(*) from areas
'''

In [9]:
# результат запроса
pd.read_sql_query(query_3, connection)


c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select count(*) from industries
'''

In [11]:
# результат запроса
pd.read_sql_query(query_3_4, connection)

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,count
0,294


***

**Выводы** *В базе данных представлены 49'197 вакансий от 23501 работодателя. География вакансий довольна обширна и представлена 1362 регионом. Диапазон сфер деятельности также довольно широк и представлен 294 направлениями*

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:

# текст запроса
query_4_1 = f'''
SELECT 
    areas.name area, 
    count(vacancies.id) cnt
FROM 
    vacancies
JOIN 
    areas ON vacancies.area_id = areas.id
GROUP BY areas.name
ORDER BY 2 DESC
LIMIT 15
'''

In [13]:
# результат запроса
df = pd.read_sql_query(query_4_1, connection)
fig = px.bar(df, 
    x='area', 
    y='cnt',
    title='Количество вакансий по регионам',
       labels ={'area':'Город','cnt':'Количество'},
       width =800,
       height=500)
fig.show()
fig.write_html("images/vacancies_cities.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
SELECT COUNT(*)
FROM vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL
'''

In [15]:
# результат запроса
pd.read_sql_query(query_4_2, connection)

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
SELECT 
    round(avg(salary_from),0) avg_low,
    round(avg(salary_to),0) avg_high
FROM vacancies
'''

In [17]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
df

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,avg_low,avg_high
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
SELECT 
    schedule,
    employment,
    count(id)
FROM vacancies
GROUP BY schedule, employment
ORDER BY 3 DESC
'''

In [19]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
fig = px.scatter(
    df,  
    x='count', 
    y= 'employment', 
    size='count', 
    color= 'schedule', 
    size_max=80,
    labels ={'count':'Количество вакансий','employment':'Занятость'},
    title= 'Распределение количества вакансий по типу трудоустройства и типу рабочего графика')
fig.show()
fig.write_html("images/shedule_employment.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
SELECT 
    experience,
    count(id)
FROM vacancies
GROUP BY experience
ORDER BY 2
'''

In [21]:
# результат запроса
df = pd.read_sql_query(query_4_5, connection)
fig = px.pie(
    df, 
    values='count', 
    names='experience', 
    width =800,
    height=500,
    title='Структура ваканий по требуемому опыту работы')
fig.show()
fig.write_html("images/experience.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



***

**Выводы:** *Распределение вакансий по регионам демонстрирует лидерство по числу вакансий крупных столичных городов и городов-миллиоников, среди которых существенно выделяется Москва с 5333 вакансиями. Данный факт легко объясняется тем обстоятельством, что в крупных городах находится большое количество работадателей. График распределения количества работодалей по регионам (ниже), практически повторяет график распределения вакансий. Около половины работодателей не указывают размер заработной платы, оставшиеся же предлагают своим соискателям в среднем от 71 до 110 тыс. руб. В основном работодатели хотят видеть своих работников на полной занятости (более 90% вакансий) и на полный рабочий день (более 70% вакансий), удаленный график также востребован, но в гораздо меньшем количестве. Вакансии на прочие типы занятости совершенно незначительны. Интересно, что в плане опыта работы, требования работодателей довольно лояльны. Практически 68% вакансий размещены для молодых специалистов без опыта, либо с опытом до 3 лет. Это говорит отом, что компании готовы и предпочитают "ростить" специалистов для себя и вкладывать в их профессиональное развитие.*

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''
SELECT 
    employers.name,
    count(vacancies.id)
FROM vacancies
JOIN employers ON vacancies.employer_id = employers.id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 15
'''

In [23]:
# результат запроса
df = pd.read_sql_query(query_5_1, connection)
fig = px.bar(df, 
    x='name', 
    y='count',
    title='Количество вакансий по работодателям',
       labels ={'name':'Работодатель','count':'Количество'},
       width =800,
       height=500)
fig.show()
fig.write_html("images/vacancies_employer.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''
SELECT 
    a.name,
    COUNT(e.id) employers_count
FROM areas a
LEFT JOIN employers e ON a.id = e.area
LEFT JOIN vacancies v ON a.id = v.area_id
GROUP BY 1
HAVING COUNT(v.id) = 0
ORDER BY 2 DESC
LIMIT 1
'''

In [25]:
# результат запроса
df = pd.read_sql_query(query_5_2, connection)
df

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name,employers_count
0,Россия,410


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''
SELECT 
    employers.name,
    COUNT(DISTINCT vacancies.area_id)
FROM employers
JOIN vacancies ON employers.id = vacancies.employer_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 15
'''

In [27]:
# результат запроса
df = pd.read_sql_query(query_5_3, connection)
fig = px.bar(df, 
    x='name', 
    y='count',
    title='Количество регионов, в которых публикуют вакансии работодатели',
       labels ={'name':'Работодатель','count':'Количество регионов'},
       width =800,
       height=500)
fig.show()
fig.write_html("images/vacancies_region_employers.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''
SELECT COUNT(*)
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''

In [29]:
# результат запроса
df = pd.read_sql_query(query_5_4, connection)
df

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''
SELECT 
    e.name
FROM employers e
LEFT JOIN employers_industries ei ON e.id = ei.employer_id
GROUP BY 1
HAVING COUNT(ei.industry_id) = 4
ORDER BY 1
OFFSET 2
LIMIT 1
'''

In [31]:
# результат запроса
result = pd.read_sql_query(query_5_5, connection)
result

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''
SELECT 
    COUNT(ei.employer_id)
FROM employers_industries ei
JOIN industries i ON ei.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''

In [33]:
# результат запроса
result = pd.read_sql_query(query_5_6, connection)
result

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [66]:
# код для получения списка городов-милионников
url  ='https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'
table = pd.read_html(url)[1]
million_cities = tuple(map(lambda x: 'Воронеж' if x[:7] == 'Воронеж' else x, table['Город'].values))

In [67]:
# текст запроса
query_5_7 = f'''
SELECT 
    a.name,
    COUNT(v.id) cnt
FROM areas a
JOIN vacancies v ON a.id = v.area_id
JOIN employers e ON v.employer_id = e.id
WHERE 
    a.name in {million_cities} AND
    e.name = 'Яндекс'
GROUP BY 1
UNION
SELECT
    'Total', 
    COUNT(v.id) 
FROM areas a
JOIN vacancies v ON a.id = v.area_id
JOIN employers e ON v.employer_id = e.id
WHERE 
    a.name in {million_cities} AND
    e.name = 'Яндекс'    
ORDER BY 2
'''

In [68]:
# результат запроса
df = pd.read_sql_query(query_5_7, connection)
df

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

**Выводы** *Среди работодателей безусловным лидером по найму персонала является компания Яндекс. Количество вакансий, размещенных от этой компании в 4 раза превышаает количество вакансий от следующего по количеству вакансий работодателя - Ростелеком. Это говорит о том, что Яндекс динамично развивается и расширяет свой бизнес, в том числе и по географии. Об этом свидетельствует тот факт, что Яндекс также является лидером по количеству регионов, в которых работодатели размещают свои вакансии. Не смотря на то, что более трети работодателей не указало сферу деятельности, анализ количества вакансий по отраслям все-таки позволяет выявить общие тенденции. График ниже демонстрирует, что наибольшее количество вакансий открыто в области Разрабоки программного обеспечения, системной интеграции и поисковых систем, т.е в ИТ сфере. Таким образом можно сделать вывод, что ИТ наиболее динамично развивающийся сектор экономики, где востребовано существенное количество персонала. Сказывается ли это как-то на уровне заработной платы. Давайте построим график распределения средней заработной платы по отраслям. Интересно, что среди верхних значений средней ЗП, отрасли ИТ сферы даже не фигурируют. Из этого можно сделать вывод, что не смотря на широкий найм со стороны ИТ компаний, все-таки существует значительное предложение со стороны соискателей и конкуренция среди них высока, что позволяет работодателям держать умеренный уровень заработной платы. Также сказывается тот факт, что, как мы убедились, в структуре вакансий ИТ отрасли существенный вес занимают вакансии с низким опытом работы, которые не предполагают высокой оплаты труда.*

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''
SELECT 
    COUNT(v.id)
FROM vacancies v
WHERE 
    LOWER(v.name) LIKE '%data%' OR
    LOWER(v.name) LIKE '%данн%'
'''

In [38]:
# результат запроса
result = pd.read_sql_query(query_6_1, connection)
result

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''
SELECT COUNT(*)
FROM vacancies v
WHERE 
   (LOWER(v.name) LIKE '%data scientist%' OR 
    LOWER(v.name) LIKE '%data science%' OR
    LOWER(v.name) LIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' and v.name not ILIKE '%HTML%') OR
    LOWER(v.name) LIKE '%machine learning%' OR
    LOWER(v.name) LIKE '%машинн%обучен%')
    AND ((v.employment = 'Стажировка') OR (v.experience = 'Нет опыта') OR LOWER(v.name) LIKE '%junior%')
'''

In [40]:
# результат запроса
result = pd.read_sql_query(query_6_2, connection)
result

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''
SELECT 
    COUNT(v.id)
FROM vacancies v
WHERE 
   (LOWER(v.name) LIKE '%data scientist%' OR 
    LOWER(v.name) LIKE '%data science%' OR
    LOWER(v.name) LIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' and v.name not ILIKE '%HTML%') OR
    LOWER(v.name) LIKE '%machine learning%' OR
    LOWER(v.name) LIKE '%машинн%обучен%') AND
    (lower(v.key_skills) LIKE '%postgres%' or lower(v.key_skills) LIKE '%sql%')
'''

In [42]:
# результат запроса
result = pd.read_sql_query(query_6_3, connection)
result

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_4 = f'''
SELECT 
    COUNT(v.id)
FROM vacancies v
WHERE 
    (LOWER(v.name) LIKE '%data scientist%' OR 
    LOWER(v.name) LIKE '%data science%' OR
    LOWER(v.name) LIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' and v.name not ILIKE '%HTML%') OR
    LOWER(v.name) LIKE '%machine learning%' OR
    LOWER(v.name) LIKE '%машинн%обучен%') AND
    LOWER(v.key_skills) LIKE '%python%'
'''

In [44]:
# результат запроса
result = pd.read_sql_query(query_6_4, connection)
result

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса
query_6_5 = f'''
SELECT round(avg(length(v.key_skills) - length(replace(v.key_skills, '\t',''))+1),2) avg_num_of_skills
FROM vacancies v
WHERE 
    LOWER(v.name) LIKE '%data scientist%' OR 
    LOWER(v.name) LIKE '%data science%' OR
    LOWER(v.name) LIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' and v.name not ILIKE '%HTML%') OR
    LOWER(v.name) LIKE '%machine learning%' OR
    LOWER(v.name) LIKE '%машинн%обучен%'
'''

In [46]:
# результат запроса
result = pd.read_sql_query(query_6_5, connection)
result

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



,avg_num_of_skills
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса
query_6_6 = f'''
SELECT 
    v.experience,
    ROUND(AVG((COALESCE(v.salary_from, v.salary_to) + COALESCE(v.salary_to, v.salary_from))/2),0) avg_salary
FROM vacancies v
WHERE (
    LOWER(v.name) LIKE '%data scientist%' OR 
    LOWER(v.name) LIKE '%data science%' OR
    LOWER(v.name) LIKE '%исследователь данных%' OR
    (v.name LIKE '%ML%' and v.name not ILIKE '%HTML%') OR
    LOWER(v.name) LIKE '%machine learning%' OR
    LOWER(v.name) LIKE '%машинн%обучен%') AND
    (v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL)    
GROUP BY v.experience
'''

In [48]:
# результат запроса
df = pd.read_sql_query(query_6_6, connection)
fig = px.bar(df, 
    x='experience', 
    y='avg_salary',
    title='Средняя ЗП, руб. в зависимости от опыта работы',
       labels ={'experience':'Опыт работы','avg_salary':'Средняя ЗП, руб.'},
       width =800,
       height=500)
fig.show()
fig.write_html("images/exp_salary.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



***

**Вывод:** *Предметный анализ вакансий, связанных с анализом данных выявил довольно странные факты, не вызывающие оптимизма среди аналитиков данных. Так, количество вакансий, связанных с обработкой данных составляет 1771 или всего около 5% от числа совокупных вакансий в трех ИТ отраслях - лидерах по числу вакансий. Что сложно как-то объяснить на фоне бурного развития технологий обработки больших данных и искусственного интеллекта. Число вакансий уровня "junior" мизерно и составляет 51 или менее 3%. Утешительным моментом может являться тот факт, что они вообще есть. Анализ требований к ключевым навыкам, предъявляемых работадалями показал, что наиболее востребовано знаяние SQL и Python (встречаются 201 и 351 раз), а в среднем работодатель хочет чтобы соискатель обладал 6-7 компетенциями. Ожидаемо уровень средней ЗП предлагаемой работодателями зависит от опыта работы и лежит в диапазоне от 75 тыс. ру. для джуниора и 243 тыс. руб. для специалиста с опытом более трех лет.*

# Общий вывод по проекту

**Вывод:**
 
* Наиболее обширный рынок труда это крупные города: столицы и города-миллионеры. В них находится наибольшее число работодателей и предлагается наибольшее количество вакансий. Среди них с существенным отрывом лидирует г. Москва
* Средний уровень заработной платы, предлагаемый работодателями лежит в диапазоне от 71 до 110 тыс. руб. Хотя данные цифры надо принимать с определенными допущениями, поскольку около половины работодателей не публикуют предлагаемую ЗП. 
* В основном работодатели хотят видеть своих работников на полной занятости (более 90% вакансий) и на полный рабочий день (более 70% вакансий), удаленный график также востребован, но в гораздо меньшем количестве. Вакансии на прочие типы занятости совершенно незначительны. 
* Требования работодателей к пыту работы довольно лояльны. Практически 68% вакансий размещены для молодых специалистов без опыта, либо с опытом до 3 лет.
* Среди работодателей безусловный лидер по найму персонала - Яндекс. Количество вакансий, размещенных от этой компании в 4 раза превышаает количество вакансий от следующего по количеству вакансий работодателя - Ростелеком. Яндекс также является лидером по количеству регионов, в которых работодатели размещают свои вакансии. 
* В разрезе отраслей экономики, наибольшее количество вакансий открыто в ИТ компаниях, не смотря на это средний уровень предлагаемой ЗП уступает оплате труда в других отраслях.
* Не смотря на то, что ИТ отрасль является лидером по количеству предлагаемых вакансий. Обработка данных не является той областью, где наблюдается дефицит кадров. Количество вакансий, в этой сфере составляет всего около 5%. Наибольший дефицит кадров наблюдается в области Разработки программного обеспечения. 
* Число вакансий уровня "junior" крайне незначительно и составляет менее 3%. 
* Основные навыки, которыми должен владеть соискатель на позицию Data Scientist - SQL и Python. 
* Уровень средней ЗП предлагаемой работодателями зависит от опыта работы и лежит в диапазоне от 75 тыс. ру. для джуниора и 243 тыс. руб. для специалиста с опытом более трех лет.

In [51]:
query_7 = f'''
SELECT 
    a.name,
    COUNT(e.id) cnt
FROM areas a
JOIN employers e ON a.id = e.area
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 15
'''
df = pd.read_sql_query(query_7, connection)
fig = px.bar(df, 
    x='name', 
    y='cnt',
    title= 'Количество работодателей по регионам',
       labels ={'name':'Регионы','cnt':'Количество'},
       width =800,
       height=500)
fig.show()
fig.write_html("images/area_employers.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



In [52]:
query_8 = f'''
SELECT 
    LEFT(i.name, 25) ind,
    COUNT(v.id) cnt
FROM industries i
JOIN employers_industries ei ON i.id = ei.industry_id
JOIN vacancies v ON ei.employer_id = v.employer_id
GROUP BY 1 
ORDER BY 2 DESC
LIMIT 12
'''
df = pd.read_sql_query(query_8, connection)
fig = px.bar(df, 
    x='ind', 
    y='cnt',
    title= 'Количество вакансий по отраслям',
       labels ={'ind':'Отрасли','cnt':'Количество'},
       width =1000,
       height=500)
fig.show()
fig.write_html("images/inustry_vacancy.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy



In [53]:
query_9 = f'''
SELECT
    LEFT(i.name, 25) ind, 
    ROUND(AVG((COALESCE(v.salary_from, v.salary_to) + COALESCE(v.salary_to, v.salary_from))/2),0) avg_salary
FROM industries i
JOIN employers_industries ei ON i.id = ei.industry_id
JOIN vacancies v ON ei.employer_id = v.employer_id
WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 15
'''
df = pd.read_sql_query(query_9, connection)
fig = px.bar(df, 
    x='ind', 
    y='avg_salary',
    title='Средняя ЗП, руб. по отраслям',
       labels ={'ind':'Отрасль','avg_salary':'Средняя ЗП, руб.'},
       width =800,
       height=500)
fig.show()
fig.write_html("images/ind_salary.html")

c:\Users\Администратор.000\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\io\sql.py:761: UserWarning:

pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy

